# Why does count get corrupted while running same code in threads?

Python has a global interpreter lock, which means that only one thread can execute bytecode instructions at an given time. That means that when, you switch to a different thread the current execution is frozen.

Consider the example below. Can you use the `dis` module to gain insight into why the counter in one of the examples gets corrupted.

In [6]:
from threading import Thread, Lock

class Counter:
    def __init__(self):
        self.count = 0
        
    def increment(self, by):
        self.count += by
        
class LockingCounter:
    def __init__(self):
        self.count = 0
        self.lock = Lock()
        
    def increment(self, by):
        self.lock.acquire()
        self.count += by
        self.lock.release()
        
def worker(s, how_many, counter):
    for _ in range(how_many):
        counter.increment(1)
        
def run_threads(func, how_many, counter):
    threads = []
    for i in range(5):
        args = (i, how_many, counter)
        thread = Thread(target=func, args=args)
        threads.append(thread)
        thread.start()
    for thread in threads:
        thread.join

how_many = 10 ** 5

# No threading
counter = Counter()
worker(0, how_many, counter)
print('Counter no threading is {}'.format(counter.count))

# Non locking example
counter = Counter()
run_threads(worker, how_many, counter)
print('Counter without locks is {}'.format(counter.count))

# Locking sample
counter = LockingCounter()
run_threads(worker, how_many, counter)
print('Counter with locks is {}'.format(counter.count))

Counter no threading is 100000
Counter without locks is 247575
Counter with locks is 61311
